In [95]:
import numpy as np 
from itertools import combinations
from tqdm import tqdm

# Cellular sheaves on graphs 
## Learning sheaf laplacian through minimum total variation approach 

### Generating a toy-case topology

In [96]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,2),
    (0,6),
    (1,3),
    (1,5),
    (2,3),
    (2,4),
    (3,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

In [97]:
d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(3,3),
        e[1]:np.random.randn(3,3)
        } 
        for e in edges
    }                                           # Incidency linear maps

In [98]:
F

{(0,
  1): {0: array([[ 0.30018377,  0.01274902, -1.23677388],
         [ 0.84354805, -0.61801721, -0.42274405],
         [ 1.2320627 , -0.03098521, -0.25735184]]), 1: array([[-0.50018527, -0.17989843, -0.22823313],
         [ 1.06078799, -0.06750924, -0.73415582],
         [-0.42407321, -0.65968976, -0.64874126]])},
 (0,
  2): {0: array([[ 0.53922588,  0.71854784, -0.16607856],
         [ 0.91763542,  0.28069313, -0.9749603 ],
         [-0.20275708, -1.41826462, -1.02604958]]), 2: array([[-0.63044338, -0.58708235, -0.15421156],
         [ 2.12742565,  1.63479133, -0.02070181],
         [ 0.49604176,  0.71034996,  0.3055147 ]])},
 (0,
  6): {0: array([[-0.51720257, -0.78004974, -1.13419251],
         [-0.24644268,  1.06929833,  0.03214937],
         [-0.0187837 , -2.16532669, -1.02754631]]), 6: array([[-0.98701016,  1.47749263,  0.49410047],
         [ 0.35574068, -0.21964233, -0.57431251],
         [-0.67214513,  0.67796445,  0.25418034]])},
 (1,
  3): {1: array([[ 0.77629891,  0.1038

In [99]:
# Graph representation

A = np.zeros((7,7))

for edge in edges:
    u = edge[0] 
    v = edge[1] 

    A[u,v] = 1
    A[v,u] = 1

D = np.diag(np.sum(A, axis = 0))
L = D - A

In [100]:
# Sheaf representation 

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

L_f = B.T @ B

In [101]:
B

array([[ 0.30018377,  0.01274902, -1.23677388,  0.50018527,  0.17989843,
         0.22823313,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.84354805, -0.61801721, -0.42274405, -1.06078799,  0.06750924,
         0.73415582,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.2320627 , -0.03098521, -0.25735184,  0.42407321,  0.65968976,
         0.64874126,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.53922588,  0.71854784, -0.16607856,  0.        

### Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [102]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [103]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [104]:
Y = U @ np.diag(H) @ U.T @ X

Now we deploy the linear map learning strategy. If we consider each of the linear maps connecting the stalks over the nodes with the stalk of the inciding edge, we have that the minimum total variation on the Laplacian can be rewritten as:

\begin{equation}
    \min_{\mathcal{F}_{u \triangleleft e}, \mathcal{F}_{v \triangleleft e}, e \in E} \frac{1}{2} \sum_{e \in E} \sum_{i=1}^N ||\mathcal{F}_{u \triangleleft e}x_u^i - \mathcal{F}_{v \triangleleft e}x_v^i||_F^2
\end{equation}

Clearly this problem can be decomposed into the contribution of each of the edges, requiring us to solve a subproblem for each of the possible edges: 

\begin{equation}
    \min_{\mathcal{F}_{u \triangleleft e}, \mathcal{F}_{v \triangleleft e}} \frac{1}{2} \sum_{i=1}^N ||\mathcal{F}_{u \triangleleft e}x_u^i - \mathcal{F}_{v \triangleleft e}x_v^i||_F^2
\end{equation}

This problem can be solved in a successive convex approximation fashion, being block-wise convex: the update equations are: 

\begin{gather}
    \hat{{\mathcal{F}}}_{u \triangleleft e} = {\mathcal{F}_{v \triangleleft e}} (X_vX_u^T) (X_uX_u^T)^{-1} \\ \nonumber
    \hat{{\mathcal{F}}}_{v \triangleleft e} = {\mathcal{F}_{u \triangleleft e}} (X_uX_v^T) (X_vX_v^T)^{-1}
\end{gather}

In the end we can compute the total energy related to each edge: this means that if we sort out all the edges with respect to this measure we can rebuild the laplacian considering the given $t_0$ number of edges and the associated linear maps. 

In [105]:
# Alternated Linear Maps Learning

def ALML(X_u, X_v, d, T = 500):
    # Initialization 

    F_u = np.random.randn(d)
    F_v = np.random.randn(d)
    gamma = 0.99

    # This matrices can be computed out of the learning loop 

    vu = X_v @ X_u.T
    uv = X_u @ X_v.T
    uu = np.linalg.inv(X_u @ X_u.T)
    vv = np.linalg.inv(X_v @ X_v.T)

    # Alternated learning through blockwise convex programs

    for _ in range(T): 
        # Local step
        F_u_hat = F_v @ vu @ uu
        F_v_hat = F_u @ uv @ vv

        # Convex smoothing
        F_u = F_u + gamma*(F_u_hat - F_u)
        F_v = F_v + gamma*(F_v_hat - F_v)

        gamma *= 0.9
        
    return F_u, F_v 

In [106]:
all_edges = list(combinations(nodes, 2))
maps = {
    e:{
        e[0] : np.zeros((3,3)), 
        e[1] : np.zeros((3,3))
        } 
    for e in all_edges
    }

energies = {
    e : 0
    for e in all_edges
    }

In [107]:
for e in tqdm(all_edges):
    u = e[0]
    v = e[1]

    X_u = Y[u*d:(u+1)*d,:]
    X_v = Y[v*d:(v+1)*d,:]

    F_u, F_v = ALML(X_u, X_v, d, T = 500)

    maps[e][u] = F_u
    maps[e][v] = F_v

    L = 0

    for i in range(100):
        x_u = X_u[:,i]
        x_v = X_v[:,i]
        L += np.linalg.norm(F_u @ x_u - F_v @ x_v)
        
    energies[e] = L

100%|██████████| 21/21 [00:00<00:00, 173.55it/s]


In [108]:
retrieved = sorted(energies.items(), key=lambda x:x[1])[:E]

In [111]:
B_hat = np.zeros((d*E, d*V))

for i in range(10):
    edge = retrieved[i][0]

    u = edge[0] 
    v = edge[1] 

    B_u = maps[edge][u]
    B_v = maps[edge][v]

    B_hat[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

L_f_hat = B_hat.T @ B_hat

In [112]:
np.linalg.norm(L_f - L_f_hat)

39.925487463471306